# Microgrid sizing optimization

(copied from Microgrid sizing optimization example notebook)

In [1]:
using Microgrids
using NLopt # optimization solvers
using Printf # pretty print results
using BenchmarkTools # for timing performance, optional

## Load Microgrid project data

Loading parameters and time series for a Microgrid project with *wind* and *solar* sources, plus a *battery* and a *dispatchable generator*. 
Values gathered from the [Microgrid_Wind-Solar.ipynb]((Microgrid_Wind-Solar.ipynb)) notebook.

In [2]:
include("../../Microgrids.jl/examples/Microgrid_Wind-Solar_data.jl");

Data definition for Microgrid with wind, solar, storage and generator...


## Setting up the cost function (criterion) to be optimized

The key coding objective is to **encapsulate** the microgrid simulators (`simulate` function of `Microgrids.jl` package) into an objective function that can be called by the optimization algorithm, that is which respects its expected interface (here NLopt).

To increase the modularity which facilites using optimization solvers others that NLopt's we implement the encapsulation by **3 nested functions**:

1. Simulation of Microgrid project described by a sizing vector `x` (vector input) → returns all simulation statistics
2. Extract KPIs of interest to build a multi-objective criterion: here lcoe and shedding rate
3. Combine these KPIs as one mono-objective criterion: here LCOE + penalty if shedding rate > shed_max
   - and match the interface expected by NLopt's optimizers

but if one cares more about compactness, this could be assembled into one single function.

### Wrapper of the Microgrid simulator

accept a sizing vector `x`, then create all `Microgrids.jl` components and call `simulate`

In [3]:
"""Simulate the performance of a Microgrid project of size `x`
with x=[power_rated_gen, energy_rated_sto, power_rated_pv, power_rated_wind]

Returns stats, costs
"""
    function simulate_microgrid(x, relax)
    project = Project(lifetime, discount_rate, timestep, "€")
    # Split decision variables (converted MW → kW):
    power_rated_gen = x[1]*1000
    energy_rated_sto = x[2]*1000
    power_rated_pv = x[3]*1000
    power_rated_wind = x[4]*1000

    # Create components
    gen = DispatchableGenerator(power_rated_gen,
        fuel_intercept, fuel_slope, fuel_price,
        investment_price_gen, om_price_gen, lifetime_gen,
        load_ratio_min,
        replacement_price_ratio, salvage_price_ratio, fuel_unit)
    batt = Battery(energy_rated_sto,
        investment_price_sto, om_price_sto, lifetime_sto, lifetime_cycles,
        charge_rate, discharge_rate, loss_factor_sto, SoC_min, SoC_ini,
        replacement_price_ratio, salvage_price_ratio)
    pv = Photovoltaic(power_rated_pv, irradiance,
        investment_price_pv, om_price_pv,
        lifetime_pv, derating_factor_pv,
        replacement_price_ratio, salvage_price_ratio)
    windgen = WindPower(power_rated_wind, cf_wind,
        investment_price_wind, om_price_wind,
        lifetime_wind,
        replacement_price_ratio, salvage_price_ratio)
    mg = Microgrid(project, Pload, gen, batt, [pv, windgen])

    # Launch simulation:
    traj, stats, costs = simulate(mg, relax)

    return stats, costs
end

simulate_microgrid

Test of the simulator wrapper (on a baseline sizing):

In [4]:
# Baseline sizing: same as in Microgrid_Wind-Solar.ipynb notebook
x_base = [power_rated_gen, energy_rated_sto, power_rated_pv, power_rated_wind]/1000.
x_base = [1707., 0., 0., 0., 0.]/1000.
# run simulation:
stats, costs = simulate_microgrid(x_base, 1.0)
x_base, costs.lcoe, costs.npc/1e6

([1.707, 0.0, 0.0, 0.0, 0.0], 0.28992486610483786, 27.68381750549201)

In [5]:
stats

OperationStats with fields:
- served_energy: 6.775e6 kWh
- shed_energy: 0.0 kWh
- shed_max: 0.0 kW
- shed_hours: 0.0 h
- shed_duration_max: 0.0 h
- shed_rate: 0.0 in [0,1]
- gen_energy: 6.775e6 kWh
- gen_hours: 3968.9 h
- gen_fuel: 1.626e6 L
- storage_cycles: NaN 
- storage_char_energy: -0.0 kWh
- storage_dis_energy: 0.0 kWh
- storage_loss_energy: -0.0 kWh
- spilled_energy: 0.0 kWh
- spilled_max: 0.0 kW
- spilled_rate: NaN in [0,1]
- renew_potential_energy: 0.0 kWh
- renew_energy: 0.0 kWh
- renew_rate: 0.0 in [0,1]


Generator cost check:

In [7]:
costs.generator 

CostFactors(2.768381750549201e7, 682800.0, 2.256690377894558e6, 1.909723569242347e6, 2.2916682830908217e7, -82079.27255311127)

Classical linear salvage: (77.6k)

- CostFactors(2.7688247701701254e7, 682800.0, 2.256690377894558e6, 1.909723569242347e6, 2.2916682830908217e7, -77649.07634386775)

economically consistent definition: larger salvage (82.0k)

- CostFactors(2.768381750549201e7, 682800.0, 2.256690377894558e6, 1.909723569242347e6, 2.2916682830908217e7, -82079.27255311127)

### Build the objective functions (criteria)

- first bi-objective function x ↦ (lcoe, shedding rate)(x)
- then wrapped into a mono objective x ↦ J(x) by using a penalty for the excess of shedding rate
  - and match the interface expected by NLopt's optimizers

In [9]:
"Multi-objective criterion for microgrid performance: lcoe, shedding rate"
function obj_multi(x)
    stats, costs = simulate_microgrid(x, 1.0) # full relaxation
    # Extract KPIs of interest
    lcoe = costs.lcoe # $/kWh
    shed_rate = stats.shed_rate; # in [0,1]
    return lcoe, shed_rate
end

obj_multi

In [10]:
"""Mono-objective criterion: LCOE + penalty if shedding rate > `shed_max`

with signature adapted to NLopt with `grad` as 2nd argument

load shedding penalty threshold `shed_max` should be in  [0,1[
"""
function obj(x, grad, shed_max, w_shed_max=1e5)
    lcoe, shed_rate = obj_multi(x)
    over_shed = shed_rate - shed_max
    if over_shed > 0.0
        penalty = w_shed_max*over_shed
    else
        penalty = 0.0
    end
    J = lcoe + penalty
end

obj

### Tests the objective functions

Sizing being tested:
- baseline sizing from the simulation notebook: perfect quality of service (QoS) with zero load shedding
- baseline modified with a halved generator sizing: very good QoS with a bit of load shedding → not penalized
- small PV and small wind generators only: low LCOE (i.e. the production-only LCOE of these sources) but but extremely bad QoS → huge penalty

In [11]:
# Test:
shed_max = 0.0 # in [0,1]

x_shed = [power_rated_gen/2, energy_rated_sto, power_rated_pv, power_rated_wind]/1000.
x_pv   = [0. 0. 500.   0.]/1000.
x_wind = [0. 0.   0. 500.]/1000.

println("Base. multi: ", obj_multi(x_base), " mono: ", obj(x_base,[], shed_max))
println("Shed. multi: ", obj_multi(x_shed), " mono: ", obj(x_shed,[], shed_max))
println("PV.   multi: ", obj_multi(x_pv), " mono: ", obj(x_pv,[], shed_max))
println("Wind. multi: ", obj_multi(x_wind), " mono: ", obj(x_wind,[], shed_max))

Base. multi: (0.28992486610483786, 0.0) mono: 0.28992486610483786
Shed. multi: (0.1965916081469468, 0.009602858175478355) mono: 960.4824091559824
PV.   multi: (0.10149685980966677, 0.923547868561659) mono: 92354.8883530257
Wind. multi: (0.10040264224635914, 0.74395737102815) mono: 74395.83750545724


## Optimization

### Setting up the optimization problem

bounds of the design space and starting point: derived from maximal load power

In [12]:
Pload_max = maximum(Pload)

xmin = [0., 0., 1e-3, 0.] # 1e-3 instead of 0.0, because LCOE is NaN if ther is exactly zero generation
x0 = [1.0, 3.0, 3.0, 2.0] * (Pload_max/1000)
xmax = [1.2, 10.0, 10.0, 5.0] * (Pload_max/1000)

4-element Vector{Float64}:
  2.0484
 17.07
 17.07
  8.535

Optionally ban some choices:

In [13]:
# Solar power forbidden (optional)
#x0[3] = 1e-3
#xmax[3] = 1e-3
# Wind power forbidden (optional)
#x0[4] = 0.
#xmax[4] = 0.

Check cost function on `xmin` and `xmax`

In [14]:
obj_multi(xmin), obj_multi(xmax)

((0.10149685980963595, 0.9998470957371233), (0.821047482642638, 0.0))

### Wrapper of the optimization process

This is an optional step, but recommended to explore easily the impact of the many parameters taken by optimization algorithms.

See [optimization termination conditions](https://nlopt.readthedocs.io/en/latest/NLopt_Introduction/#termination-conditions) in NLopt documention for the meaning of `xtol_rel`

In [15]:
"""Optimize sizing of microgrid based on the `obj` function

Parameters:
- `x0`: initial sizing (for the algorithms which need them)
- `shed_max`: load shedding penalty threshold (same as in `obj`)
- `algo` could be one of LN_SBPLX, GN_DIRECT, GN_ESCH...
- `maxeval`: maximum allowed number of calls to the objective function,
  that is to the microgrid simulation
- `xtol_rel`: termination condition based on relative change of sizing, see NLopt doc.
- `srand`: random number generation seed (for algorithms which use some stochastic search)

Problem bounds are taken as the global variables `xmin`, `xmax`,
but could be added to the parameters as well.
"""
function optim_mg(x0, shed_max, algo=:LN_SBPLX, maxeval=1000, xtol_rel=1e-4, srand=1)
    nx = length(x0) # number of optim variables
    opt = Opt(algo, nx)
    NLopt.srand(srand)
    
    opt.lower_bounds = xmin
    opt.upper_bounds = xmax
    opt.min_objective = (x, grad) -> obj(x, grad, shed_max)
    opt.xtol_rel = xtol_rel
    opt.maxeval = maxeval
    
    (fopt, xopt, ret) = optimize(opt, x0)
    return xopt, ret, opt.numevals
end

optim_mg

### Run optimization & analyze results

In [46]:
algo = :GN_CRS2_LM # could be one of GN_CRS2_LM, GN_DIRECT, GN_ESCH, LN_SBPLX...
shed_max = 0.00 # in [0,1]
maxeval=10000
srand=1
xopt, ret, numevals = optim_mg(x0, shed_max, algo, maxeval, 1e-5, srand)

@printf("%s algo: %s after %d iterations. \nx*=", algo, ret, numevals)
println(round.(xopt*1000; digits=1)) # kW
lcoe_opt, shed_rate_opt = obj_multi(xopt)
println("LCOE*: ", lcoe_opt)
println("shed*: ", shed_rate_opt)

GN_CRS2_LM algo: XTOL_REACHED after 2216 iterations. 
x*=[1573.9, 970.9, 1169.6, 1293.8]
LCOE*: 0.18122665766333945
shed*: 0.0


optional local "polishing":

In [47]:
algo_polish = :LN_SBPLX
xopt_polish, ret, numevals = optim_mg(xopt, shed_max, algo_polish, maxeval, 1e-5)

@printf("%s polish: %s after %d iterations. \nx*=", algo_polish, ret, numevals)
println(round.(xopt_polish*1000; digits=1)) # kW
lcoe_opt, shed_rate_opt = obj_multi(xopt_polish)
println("LCOE*: ", lcoe_opt)
println("shed*: ", shed_rate_opt)

LN_SBPLX polish: XTOL_REACHED after 207 iterations. 
x*=[1573.9, 971.0, 1169.6, 1293.8]
LCOE*: 0.18122664580933595
shed*: 0.0


Retrieve all performance statistics of the optimized sizing

In [52]:
xopt = [ # xopt_polish
    1.5739391977120336
    0.9709769074394994
    1.1696385309544288
    1.2938324399374947]

4-element Vector{Float64}:
 1.5739391977120336
 0.9709769074394994
 1.1696385309544288
 1.2938324399374947

In [53]:
stats_opt, costs_opt = simulate_microgrid(xopt, 1.0)
costs_opt.npc

1.7304639843841173e7

In [54]:
stats_opt, costs_opt = simulate_microgrid(xopt, 1.0)
@printf("NPC*: %.2f M\$ (compared to %.2f M\$ in baseline)\n", costs_opt.npc/1e6, costs.npc/1e6)
@printf("rate of renewables: %.1f %%\n", stats_opt.renew_rate*100)
# Display all operation statistics
stats_opt

NPC*: 17.30 M$ (compared to 27.68 M$ in baseline)
rate of renewables: 69.5 %


OperationStats with fields:
- served_energy: 6.775e6 kWh
- shed_energy: 0.0 kWh
- shed_max: 0.0 kW
- shed_hours: 0.0 h
- shed_duration_max: 0.0 h
- shed_rate: 0.0 in [0,1]
- gen_energy: 2.0691e6 kWh
- gen_hours: 1314.6 h
- gen_fuel: 496570.0 L
- storage_cycles: 200.0 
- storage_char_energy: 203910.0 kWh
- storage_dis_energy: 184490.0 kWh
- storage_loss_energy: 19420.0 kWh
- spilled_energy: 985860.0 kWh
- spilled_max: 1568.6 kW
- spilled_rate: 0.17262 in [0,1]
- renew_potential_energy: 5.7112e6 kWh
- renew_energy: 4.7253e6 kWh
- renew_rate: 0.6946 in [0,1]


Cost decomposition of the generator:

In [55]:
costs_opt.generator.investment + costs_opt.generator.replacement + costs_opt.generator.salvage,
costs_opt.generator.om, costs_opt.generator.fuel

(1.03922302139769e6, 583224.5749792828, 6.99869489975141e6)

#### *Original results* (no relax) with perfect quality of service (QoS)

bigger battery:

```
GN_CRS2_LM algo: MAXEVAL_REACHED after 1002 iterations. 
x*=[1561.8, 2821.2, 1758.3, 1464.5]
LCOE*: 0.2074272180878332
shed*: 0.0
```

In [15]:
algo = :GN_CRS2_LM # could be one of LN_SBPLX, GN_DIRECT, GN_CRS2_LM, GN_ESCH...
shed_max = 0.00 # in [0,1]
maxeval=1000
xopt, ret, numevals = optim_mg(x0, shed_max, algo, maxeval)

@printf("%s algo: %s after %d iterations. \nx*=", algo, ret, numevals)
println(round.(xopt*1000; digits=1)) # kW
lcoe_opt, shed_rate_opt = obj_multi(xopt)
println("LCOE*: ", lcoe_opt)
println("shed*: ", shed_rate_opt)

GN_CRS2_LM algo: MAXEVAL_REACHED after 1002 iterations. 
x*=[1561.8, 2821.2, 1758.3, 1464.5]
LCOE*: 0.2074272180878332
shed*: 0.0
